In [2]:
# depends on g2p-en
from datasets import load_dataset
from g2p_en import G2p

g2p = G2p()

generics_kb = load_dataset(
    'community-datasets/generics_kb', name='generics_kb_best', split='train')
ponyspeech_dataset = load_dataset(
    'synthbot/pony-speech', split='train')

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [54]:
horsewords = {}
with open('horsewords.clean') as f:
    while line := f.readline():
        spl = line.split('  ')
        horsewords[spl[0]] = spl[1]

def phonemize(text):
    spl = text.split()
    l = ''
    for s in spl:
        p = [arp for arp in g2p(s) if arp != ' ']
        arpabet_string = ''.join(p)
        l += arpabet_string + ' '
    return l.strip()
print(phonemize('But one on one, let\'s clean it.'))

BAH1T WAH1N AA1N WAH1N, LEH1TS KLIY1N IH1T.


In [55]:
from unidecode import unidecode
from tqdm import tqdm
n = 240000
corpus_output_txt = 'g2pen_corpus_ponyonly.txt'
with open(corpus_output_txt, 'w', encoding='utf-8') as f:
    #for text in tqdm(generics_kb.shuffle().select(
    #    range(n)
    #)['generic_sentence'], desc='generic_kb wiki'):
    #    f.write(phonemize(text)+'\n')
    for text in tqdm(ponyspeech_dataset['transcription'], desc='ponyspeech'):
        f.write(phonemize(text)+'\n')

ponyspeech: 100%|██████████| 64783/64783 [04:42<00:00, 229.04it/s]


In [56]:
from tokenizers import (decoders, models, normalizers,
    pre_tokenizers, processors, trainers, Tokenizer)

import re

special_tokens = ['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]']
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
    [
    pre_tokenizers.Metaspace('▁'),
    pre_tokenizers.Punctuation()
    ])
trainer = trainers.BpeTrainer(vocab_size=384,
    special_tokens=special_tokens, unk_token='[UNK]')
tokenizer.train(['g2pen_corpus_ponyonly.txt'], trainer=trainer)

In [7]:
from transformers import AutoTokenizer
parler_tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")

In [58]:
test_texts = [
  'I need some money, can I get some money? I need one million.',
    'Raise up, get yourself together, and drive that funky soul.',
    'Our results demonstrate high fidelity speech generation and a diverse range of accents',
    'Equestria the brave Equestrian.',
    'Dead nirik storage.',
    'Well, one on one, let\'s clean it!',
    'Amazon delivers packages quickly across the United States!',
    'Bitcoin and Ethereum are popular cryptocurrencies.',
    'Text-to-speech models 1 $200',
    'Queue queuing'
    ]
for test_text in test_texts:
    #print(tokenizer.pre_tokenizer.pre_tokenize_str(phonemize(test_text)))
    encoding = tokenizer.encode(phonemize(test_text))
    print(encoding.tokens)
    print(len(encoding.tokens))
    print(any([tok == '[UNK]' for tok in encoding.tokens]))

    print(parler_tokenizer.tokenize(test_text))
    print(len(parler_tokenizer(test_text).input_ids))

['▁AY1', '▁NIY1D', '▁SAH1M', '▁M', 'AH1', 'NIY0', ',', '▁KAE1N', '▁AY1', '▁GEH1T', '▁SAH1M', '▁M', 'AH1', 'NIY0', '?', '▁AY1', '▁NIY1D', '▁WAH1N', '▁M', 'IH1L', 'Y', 'AH0N', '.']
23
False
['▁I', '▁need', '▁some', '▁money', ',', '▁can', '▁I', '▁get', '▁some', '▁money', '?', '▁I', '▁need', '▁one', '▁million', '.']
17
['▁R', 'EY1', 'Z', '▁AH1P', ',', '▁GEH1T', '▁Y', 'ER0', 'S', 'EH1L', 'F', '▁T', 'AH0', 'G', 'EH1D', 'HER0', ',', '▁AH0ND', '▁DR', 'AY1', 'V', '▁DHAE1T', '▁F', 'AH1', 'NG', 'KIY0', '▁SOW1', 'L', '.']
29
False
['▁R', 'aise', '▁up', ',', '▁get', '▁yourself', '▁together', ',', '▁and', '▁drive', '▁that', '▁fun', 'ky', '▁soul', '.']
16
['▁AW1ER0', '▁RIH0', 'Z', 'AH1', 'L', 'T', 'S', '▁D', 'EH1M', 'AH0N', 'ST', 'R', 'EY2', 'T', '▁HH', 'AY1', '▁F', 'AH0D', 'EH1L', 'AH0TIY0', '▁SP', 'IY1', 'CH', '▁JH', 'EH2', 'N', 'ER0', 'EY1', 'SHAH0N', '▁AH0ND', '▁AH0', '▁D', 'A', 'Y0', 'V', 'ER1', 'S', '▁REY1N', 'JH', '▁AH1V', '▁AE1', 'KS', 'EH0', 'NT', 'S']
45
False
['▁Our', '▁results', '▁demonst

In [59]:
test_texts = [
    "The phthisic sphinx stealthily eschews the mnemonic pneumatic knickknacks.",
    "Szczecin's squirrel chirped rhythmically while munching on zwitterionic quetzal quills.",
    "Bjorn's fjord-dwelling lynx phthalate psyche was quite a conundrum.",
    "The zeitgeist of Györgyike's csárdás xylophone quintet was truly unique.",
    "Tsar Xylophone's czarina whispered žuželjka while eating sauerkraut in Århus.",
    "Queuing for Llanfairpwllgwyngyllgogerychwyrndrobwllllantysiliogogogoch's onomatopoeic eisteddfod.",
    "The kleptomaniac's schwannoma diagnosis left the psychiatrist nonplussed.",
    "Xerxes' xiphosura exhibited xenophobic behavior towards Zsa Zsa's Zhivago zhuzh.",
    "The phlegmatic pharmacist's emphysema was exacerbated by Phthonus' chthonic chrysanthemums.",
    "Kwyjibo's syzygy with Przybyszewski's psyche caused a hubbub in Ouagadougou.",
    "The bourgeois faux pas at the rendezvous caused quite a brouhaha amongst the hoi polloi.",
    "Nietzsche's Übermensch theory clashed with Schrödinger's quantum doppelgänger hypothesis.",
    "The pneumonoultramicroscopicsilicovolcanoconiosis diagnosis flummoxed the otorhinolaryngologist.",
    "Gnocchi-loving Giacomo's pince-nez glinted as he perused Goethe's oeuvre on the Champs-Élysées.",
    "The tsktsk of the ptarmigan's wings echoed through the cwtch as it flew over Loch Ness."
    ]
for test_text in test_texts:
    #print(tokenizer.pre_tokenizer.pre_tokenize_str(phonemize(test_text)))
    encoding = tokenizer.encode(phonemize(test_text))
    print(encoding.tokens)
    print(len(encoding.tokens))
    print(any([tok == '[UNK]' for tok in encoding.tokens]))

    print(parler_tokenizer.tokenize(test_text))
    print(len(parler_tokenizer(test_text).input_ids))

['▁DHAH0', '▁F', 'AH0', 'TH', 'IH1S', 'IH0K', '▁S', 'F', 'IH1NG', 'KS', '▁ST', 'EH1L', 'TH', 'IY0', 'LIY0', '▁', 'EH0', 'S', 'CH', 'UW1', 'Z', '▁DHAH0', '▁N', 'IH0', 'M', 'AA1', 'N', 'IH0K', '▁N', 'UW', '0', 'M', 'AE1', 'T', 'IH0K', '▁N', 'IH1K', 'N', 'AE', '2', 'KS', '.']
42
False
['▁The', '▁', 'p', 'h', 'this', 'ic', '▁', 's', 'phin', 'x', '▁steal', 'thi', 'ly', '▁', 'e', 'sche', 'w', 's', '▁the', '▁', 'm', 'nem', 'onic', '▁pneu', 'matic', '▁', 'k', 'nick', 'k', 'n', 'ack', 's', '.']
34
['▁SHIY1', 'S', 'IH0N', 'Z', '▁SK', 'W', 'ER1', 'AH0L', '▁CH', 'ER1', 'P', 'T', '▁R', 'IH1D', 'H', 'M', 'IH0K', 'LIY0', '▁WAY1', 'L', '▁M', 'AH1N', 'CH', 'IH0NG', '▁AA1N', '▁', 'Z', 'W', 'IH2', 'T', 'ER0', 'SH', 'IY1', 'N', 'IH0NG', '▁K', 'W', 'EH1T', 'Z', 'AH0L', '▁K', 'W', 'IH1L', 'Z', '.']
45
False
['▁S', 'z', 'c', 'zeci', 'n', "'", 's', '▁squirrel', '▁', 'chir', 'ped', '▁rhythm', 'ically', '▁while', '▁mun', 'ching', '▁on', '▁', 'z', 'wit', 'terio', 'nic', '▁qu', 'etz', 'al', '▁qui', 'll', 's', '.'

In [149]:
tokenizer.save('tokenizer_g2pen.json')

In [151]:
from transformers import PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_g2pen.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]"
)
wrapped_tokenizer.save_pretrained('tokenizer_g2pen')

('tokenizer_g2pen\\tokenizer_config.json',
 'tokenizer_g2pen\\special_tokens_map.json',
 'tokenizer_g2pen\\tokenizer.json')

In [155]:
print(wrapped_tokenizer.vocab_size)

384
